In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from collections import OrderedDict

### Importando tabelas

In [21]:
df_matriculas = pd.DataFrame()

# importando notas de 2010 a 2017
for i in range(10,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('data_science/matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])

# obtendo o id do curso de ciências e tecnologia
df_cursos = pd.read_csv('data_science/cursos-graduacao.csv', sep=';')
id_cet = df_cursos[df_cursos.nome == 'CIÊNCIAS E TECNOLOGIA'].id_curso
id_cet

20101
20102
20111
20112
20121
20122
20131
20132
20141
20142
20151
20152
20161
20162
20171
20172


17    10320810
Name: id_curso, dtype: int64

In [23]:
# filtra tabela pelo id_curso
df_matriculas = df_matriculas[df_matriculas.id_curso == float(id_cet)]
df_matriculas.id_curso.value_counts()

10320810.0    910908
Name: id_curso, dtype: int64

In [61]:
df_turmas = pd.read_csv('data_science/turmas-2010-2017-ok.csv')
#df_turmasFilt = df_turmas[(df_turmas['ano'] >= 2014) & (df_turmas['nivel_ensino'] == 'GRADUAÇÃO')]
df_turmasFilt = df_turmas[df_turmas['nivel_ensino'] == 'GRADUAÇÃO']
df_turmasFilt.ano.value_counts()

/home/andressa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2014    22531
2015    22467
2017    22309
2016    21946
2013    19799
2012    17161
2011    15786
2010    15092
Name: ano, dtype: int64

In [62]:
df_componentes = pd.read_csv('data_science/componentes-curriculares-presenciais-ok.csv')

In [63]:
# print(df_turmasFilt.columns)
# print(df_componentes.columns)

### Merges

In [64]:
df_turmasNome = pd.merge(df_turmasFilt, df_componentes, left_on='id_componente_curricular', right_on='id_componente')

In [66]:
colunas = ['id_turma', 'codigo', 'nome', 'pre_requisito', 'ano', 'periodo', 'unidade_responsavel']
df_turmasNome = df_turmasNome[colunas]

df_auxAll = pd.merge(df_matriculas, df_turmasNome, on='id_turma')
colunas2 = ['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome', 'ano', 'periodo','unidade_responsavel', 'id_curso']
df_auxAll = df_auxAll[colunas2].drop_duplicates().dropna()

In [76]:
# conjunto de alunos que não serão analisados
series_Antes2014 = df_auxAll[df_auxAll['ano']<2015].discente
# selecionando apenas alunos que entraram a partir de 2014
df_depois2014 = df_auxAll[~df_auxAll['discente'].isin(series_Antes2014)]
df_depois2014 = df_depois2014[df_depois2014['unidade_responsavel'] == 'ESCOLA DE CIÊNCIAS E TECNOLOGIA']

In [77]:
# verificando
series_Antes2014[series_Antes2014.isin(df_depois2014.discente)]

Series([], Name: discente, dtype: object)

In [108]:
colunas3 = ['codigo', 'nome', 'ano', 'periodo']
df_exportacao = df_depois2014[colunas3].drop_duplicates()
df_exportacao.to_csv('df_depois2014.csv')

In [109]:
disciplinas = ['CÁLCULO I','CÁLCULO II','CÁLCULO III','VETORES E GEOMETRIA ANALÍTICA','PRÉ-CÁLCULO','PROBABILIDADE E ESTATÍSTICA',
               'INTRODUÇÃO À FÍSICA CLÁSSICA I','INTRODUÇÃO À FÍSICA CLÁSSICA II','INTRODUÇÃO À FÍSICA CLÁSSICA III','LÓGICA DE PROGRAMAÇÃO',
               'LINGUAGEM DE PROGRAMAÇÃO','COMPUTAÇÃO NUMÉRICA','QUÍMICA GERAL','ÁLGEBRA LINEAR','MECÂNICA DOS SÓLIDOS','MECÂNICA DOS FLUIDOS',
               'CIÊNCIA E TECNOLOGIA DOS MATERIAIS']

In [110]:
df_auxTurmas = df_depois2014[df_depois2014['nome'].isin(disciplinas)]

### Correlação

In [116]:
# apenas os aprovados
df_auxAprovados = df_auxTurmas[df_auxTurmas['descricao'].str.contains('APROVADO')]
df_auxAprovados.columns

Index(['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome',
       'ano', 'periodo', 'unidade_responsavel', 'id_curso'],
      dtype='object')

In [117]:
df_auxAprovados.descricao.value_counts()

APROVADO POR NOTA    11473
APROVADO              6360
Name: descricao, dtype: int64

In [118]:
discentes= df_auxAprovados.discente.unique()

In [119]:
df_auxAprovados.columns

Index(['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome',
       'ano', 'periodo', 'unidade_responsavel', 'id_curso'],
      dtype='object')

In [120]:
df_auxAprovados.descricao.value_counts()

APROVADO POR NOTA    11473
APROVADO              6360
Name: descricao, dtype: int64

In [121]:
df_calc1 = df_auxAprovados[df_auxAprovados['nome']=='CÁLCULO I']
df_calc1 = df_calc1[['discente','media_final']]
print(df_calc1.shape)

df_calc2 = df_auxAprovados[df_auxAprovados['nome']=='CÁLCULO II']
df_calc2 = df_calc2[['discente','media_final']]
print(df_calc2.shape)

df_calc3 = df_auxAprovados[df_auxAprovados['nome']=='CÁLCULO III']
df_calc3 = df_calc3[['discente','media_final']]
print(df_calc3.shape)

df_vga = df_auxAprovados[df_auxAprovados['nome']=='VETORES E GEOMETRIA ANALÍTICA']
df_vga = df_vga[['discente','media_final']]
print(df_vga.shape)

df_pc = df_auxAprovados[df_auxAprovados['nome']=='PRÉ-CÁLCULO']
df_pc = df_pc[['discente','media_final']]
print(df_pc.shape)

df_pe = df_auxAprovados[df_auxAprovados['nome']=='PROBABILIDADE E ESTATÍSTICA']
df_pe = df_pe[['discente','media_final']]
print(df_pe.shape)

df_f1 = df_auxAprovados[df_auxAprovados['nome']=='INTRODUÇÃO À FÍSICA CLÁSSICA I']
df_f1 = df_f1[['discente','media_final']]
print(df_f1.shape)

df_f2 = df_auxAprovados[df_auxAprovados['nome']=='INTRODUÇÃO À FÍSICA CLÁSSICA II']
df_f2 = df_f2[['discente','media_final']]
print(df_f2.shape)

df_f3 = df_auxAprovados[df_auxAprovados['nome']=='INTRODUÇÃO À FÍSICA CLÁSSICA III']
df_f3 = df_f3[['discente','media_final']]
print(df_f3.shape)

df_lop = df_auxAprovados[df_auxAprovados['nome']=='LÓGICA DE PROGRAMAÇÃO']
df_lop = df_lop[['discente','media_final']]
print(df_lop.shape)

df_lp = df_auxAprovados[df_auxAprovados['nome']=='LINGUAGEM DE PROGRAMAÇÃO']
df_lp = df_lp[['discente','media_final']]
print(df_lp.shape)

df_cn = df_auxAprovados[df_auxAprovados['nome']=='COMPUTAÇÃO NUMÉRICA']
df_cn = df_cn[['discente','media_final']]
print(df_cn.shape)

df_qg = df_auxAprovados[df_auxAprovados['nome']=='QUÍMICA GERAL']
df_qg = df_qg[['discente','media_final']]
print(df_qg.shape)

df_al = df_auxAprovados[df_auxAprovados['nome']=='ÁLGEBRA LINEAR']
df_al = df_al[['discente','media_final']]
print(df_al.shape)

df_ms = df_auxAprovados[df_auxAprovados['nome']=='MECÂNICA DOS SÓLIDOS']
df_ms = df_ms[['discente','media_final']]
print(df_ms.shape)

df_mf = df_auxAprovados[df_auxAprovados['nome']=='MECÂNICA DOS FLUIDOS']
df_mf = df_mf[['discente','media_final']]
print(df_mf.shape)

df_ctm = df_auxAprovados[df_auxAprovados['nome']=='CIÊNCIA E TECNOLOGIA DOS MATERIAIS']
df_ctm = df_ctm[['discente','media_final']]
print(df_ctm.shape)

(1499, 2)
(964, 2)
(675, 2)
(1626, 2)
(1920, 2)
(953, 2)
(978, 2)
(927, 2)
(694, 2)
(1286, 2)
(943, 2)
(618, 2)
(1933, 2)
(994, 2)
(548, 2)
(559, 2)
(716, 2)


In [128]:
df_correr = pd.DataFrame()
df_correr['discente'] = discentes

disciplinas = ['ÁLGEBRA LINEAR','CÁLCULO I','CÁLCULO II','CÁLCULO III','PROBABILIDADE E ESTATÍSTICA',
               'LINGUAGEM DE PROGRAMAÇÃO','COMPUTAÇÃO NUMÉRICA','MECÂNICA DOS SÓLIDOS','MECÂNICA DOS FLUIDOS',
               'CIÊNCIA E TECNOLOGIA DOS MATERIAIS','VETORES E GEOMETRIA ANALÍTICA','PRÉ-CÁLCULO',
               'INTRODUÇÃO À FÍSICA CLÁSSICA I','INTRODUÇÃO À FÍSICA CLÁSSICA II','INTRODUÇÃO À FÍSICA CLÁSSICA III','LÓGICA DE PROGRAMAÇÃO',
               'QUÍMICA GERAL']

dataFrames = [df_al,df_calc1,df_calc2,df_calc3,df_pe,df_lp,df_cn,df_ms,df_mf,df_ctm,df_vga,df_pc,df_f1,df_f2,df_f3,df_lop,df_qg]

# dataFrames,disciplinas
for i in range(0, len(dataFrames)):
    df_correr = pd.merge(df_correr, dataFrames[i], on='discente')
    df_correr = df_correr.rename(columns={"media_final": disciplinas[i]})
    print(disciplinas[i])
#     display(df_correr.head())
    
df_correr.head()

ÁLGEBRA LINEAR
CÁLCULO I
CÁLCULO II
CÁLCULO III
PROBABILIDADE E ESTATÍSTICA
LINGUAGEM DE PROGRAMAÇÃO
COMPUTAÇÃO NUMÉRICA
MECÂNICA DOS SÓLIDOS
MECÂNICA DOS FLUIDOS
CIÊNCIA E TECNOLOGIA DOS MATERIAIS
VETORES E GEOMETRIA ANALÍTICA
PRÉ-CÁLCULO
INTRODUÇÃO À FÍSICA CLÁSSICA I
INTRODUÇÃO À FÍSICA CLÁSSICA II
INTRODUÇÃO À FÍSICA CLÁSSICA III
LÓGICA DE PROGRAMAÇÃO
QUÍMICA GERAL


,discente,ÁLGEBRA LINEAR,CÁLCULO I,CÁLCULO II,CÁLCULO III,PROBABILIDADE E ESTATÍSTICA,LINGUAGEM DE PROGRAMAÇÃO,COMPUTAÇÃO NUMÉRICA,MECÂNICA DOS SÓLIDOS,MECÂNICA DOS FLUIDOS,CIÊNCIA E TECNOLOGIA DOS MATERIAIS,VETORES E GEOMETRIA ANALÍTICA,PRÉ-CÁLCULO,INTRODUÇÃO À FÍSICA CLÁSSICA I,INTRODUÇÃO À FÍSICA CLÁSSICA II,INTRODUÇÃO À FÍSICA CLÁSSICA III,LÓGICA DE PROGRAMAÇÃO,QUÍMICA GERAL
0,d00171e04d77a7994414014d941baf8c,7.6,10.0,5.7,6.8,6.7,5.3,6.4,5.0,7.7,6.9,6.5,7.2,7.1,6.0,7.6,6.3,6.9
1,3ff085f94a31cdddd21d355d0d8f6438,8.4,10.0,8.0,6.4,7.8,6.7,8.1,6.3,9.0,7.1,8.1,8.8,10.0,7.0,10.0,9.5,8.1
2,bb7bc1c9d40935fe8d7d76f83d39ee62,10.0,10.0,8.7,8.8,10.0,9.9,8.7,8.3,10.0,7.8,6.1,7.7,10.0,8.0,10.0,9.1,6.6
3,db6560af2b1538425add4e182fe0927e,9.8,10.0,7.5,8.4,8.6,7.0,7.0,6.7,9.9,5.8,5.3,7.5,10.0,7.2,9.5,8.7,5.9
4,e749444b410bab4bf5c2a86bffaee6c7,10.0,9.3,8.2,7.0,7.5,6.9,7.8,6.0,9.5,7.0,5.3,8.5,10.0,7.6,10.0,8.0,7.0


In [129]:
df_correr.corr()

,ÁLGEBRA LINEAR,CÁLCULO I,CÁLCULO II,CÁLCULO III,PROBABILIDADE E ESTATÍSTICA,LINGUAGEM DE PROGRAMAÇÃO,COMPUTAÇÃO NUMÉRICA,MECÂNICA DOS SÓLIDOS,MECÂNICA DOS FLUIDOS,CIÊNCIA E TECNOLOGIA DOS MATERIAIS,VETORES E GEOMETRIA ANALÍTICA,PRÉ-CÁLCULO,INTRODUÇÃO À FÍSICA CLÁSSICA I,INTRODUÇÃO À FÍSICA CLÁSSICA II,INTRODUÇÃO À FÍSICA CLÁSSICA III,LÓGICA DE PROGRAMAÇÃO,QUÍMICA GERAL
ÁLGEBRA LINEAR,1.000000,0.433795,0.425757,0.465317,0.469358,0.392265,0.564618,0.416774,0.372698,0.404214,0.458072,0.374957,0.621686,0.313247,0.490992,0.421373,0.370020
CÁLCULO I,0.433795,1.000000,0.354852,0.465612,0.352516,0.119178,0.394373,0.346089,0.272338,0.325441,0.357590,0.335003,0.397867,0.357582,0.499879,0.133515,0.362826
CÁLCULO II,0.425757,0.354852,1.000000,0.437824,0.414472,0.484600,0.545521,0.499678,0.247277,0.476849,0.403261,0.332390,0.446421,0.475325,0.444758,0.343138,0.450642
CÁLCULO III,0.465317,0.465612,0.437824,1.000000,0.306652,0.353716,0.442877,0.449628,0.221049,0.412297,0.396149,0.301406,0.421308,0.503243,0.483243,0.311872,0.335718
PROBABILIDADE E ESTATÍSTICA,0.469358,0.352516,0.414472,0.306652,1.000000,0.361741,0.463498,0.464016,0.247119,0.483020,0.155370,0.198657,0.447477,0.339443,0.378946,0.358408,0.303095
LINGUAGEM DE PROGRAMAÇÃO,0.392265,0.119178,0.484600,0.353716,0.361741,1.000000,0.389952,0.421765,0.078339,0.328929,0.394587,0.310367,0.351418,0.366190,0.205008,0.582297,0.339914
COMPUTAÇÃO NUMÉRICA,0.564618,0.394373,0.545521,0.442877,0.463498,0.389952,1.000000,0.519354,0.414012,0.611588,0.411261,0.328167,0.519931,0.516695,0.532274,0.379502,0.461438
MECÂNICA DOS SÓLIDOS,0.416774,0.346089,0.499678,0.449628,0.464016,0.421765,0.519354,1.000000,0.257401,0.597981,0.306049,0.298651,0.351099,0.541892,0.465036,0.340282,0.387824
MECÂNICA DOS FLUIDOS,0.372698,0.272338,0.247277,0.221049,0.247119,0.078339,0.414012,0.257401,1.000000,0.309607,0.081569,0.205421,0.301897,0.221322,0.379098,0.254575,0.145208
CIÊNCIA E TECNOLOGIA DOS MATERIAIS,0.404214,0.325441,0.476849,0.412297,0.483020,0.328929,0.611588,0.597981,0.309607,1.000000,0.426663,0.274394,0.370984,0.456305,0.503030,0.280070,0.422189
